#Lyne and Hollick Baseflow filter
This metric measures the Base Flow Index (BFI), the ratio of Baseflow/Stream Flow. As the baseflow has already undergone division by site specific streamflow, it was not measured as a function of catchment inflows. Baseflow was calculated as defined by Ladson, A. R., Brown, R. Neal, B. and Nathan, R. (2013) in A standard approach to baseflow separation using the Lyne and Hollick filter. An increase in BFI does not indicate a higher volume of baseflow, rather a higher proportion of baseflow relative to flow. For this reason, it is more common to see (counter-intuitively) high BFIs under dry conditions.

Code associated with the paper Ladson, A. R., Brown, R. Neal, B. and Nathan, R. (2013) A standard approach to baseflow separation using the Lyne and Hollick filter. Australian Journal of Water Resources 17(1): 25-34.
  
## Inputs: 
[Gauges of interest for baseflows](https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/d05b82a7-301f-4b2b-b9bc-7a968487916a)


## Outputs:
[Results](https://data.gov.au/data/dataset/hydrologic-indicator-results-for-the-basin-plan-evaluation-2020)

In [0]:
import pandas as pd
import numpy as np
import scipy.stats 
import warnings 
warnings.filterwarnings('ignore')

## Load gauges of interest for baseflow metric 
Gets gauges of interest and their associated gauge data.

Baseflow gauge information can be found: https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/d05b82a7-301f-4b2b-b9bc-7a968487916a

In [0]:
Data = pd.read_csv("https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/d05b82a7-301f-4b2b-b9bc-7a968487916a/download/observedflows_baseflowsandflowthresholds.csv", header=None)
Data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,Condamine-Balonne,Gwydir,Namoi,Overall North,Border Rivers,Campaspe,Goulburn-Broken,Lachlan,Loddon,Overall North,Macquarie-Castlereagh,Macquarie-Castlereagh,Macquarie-Castlereagh,Overall South,Overall South,Warrego,Warrego,Murrumbidgee
1,NaN,Brenda on Culgoa,Yarraman Bridge,Bugilbone,Bourke,Mungindi,Rochester,McCoys Bridge,Booligal Weir,Kerang Weir,Weir 32,Marebone Break,Marebone Break,Warren Weir,Wentworth (lock 10),D/S Yarrawonga,Cunnamulla Weir,Wyandra,Narrandera
2,Date,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,421088,421004,425010,409025,423202CO,423203A,410005
3,8/09/1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9/09/1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform gauge data 

Organising dataframe to get it ready for analysis:
- Putting gauge numbers as column headings
- stripping header information and using this data to filter to only the locations of interest
- Adding Water Year

In [0]:
DataFrame = Data.loc[3:]
DataFrame.columns = map(str.strip,
                           Data.loc[2].astype(str).tolist())
DataFrame['Date'] = pd.to_datetime(DataFrame['Date'],
        format='%d/%m/%Y')
DataFrame.set_index('Date')

#combining sites 421090 and 421088 as they are represented as a single site in the model 
DataFrame['421090'] = pd.to_numeric(DataFrame['421090']) \
    + pd.to_numeric(DataFrame['421088'])
DataFrame = DataFrame.drop(['421088'], axis=1)
DataFrame.head()

,Date,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,421004,425010,409025,423202CO,423203A,410005
3,1953-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1953-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1953-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,653.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1953-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1953-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,922.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def waterYear(date):
  '''Takes in date,
  changes year to water year
  returns water year'''
  if date.month <= 6:  # for months Jan to Jun move them to the previous water year
      waterYear = date.year - 1
  else:

       # for months after Jun move them to this  water year

      waterYear = date.year
  return int(waterYear)

In [0]:
DataFrame['water year'] = DataFrame.apply(lambda row: \
        waterYear(row['Date']), axis=1)
DataFrame.head()


,Date,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,421004,425010,409025,423202CO,423203A,410005,water year
3,1953-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953
4,1953-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953
5,1953-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,653.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953
6,1953-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953
7,1953-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,922.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953


## Isolating baseflow component

Filter data to dates of interest (after the 1994 cap on diversions was introduced)

Isolate the baseflows in flow timeseries using the Lyne and Hollick filter functions.

In [0]:
DataFrame.index = pd.to_datetime(DataFrame.index,
                                    format='%Y-%m-%d')
DataFrame = DataFrame.reset_index(drop=True)
DataFrame = DataFrame[DataFrame['water year'] >= 1994]
DataFrame = DataFrame.drop(columns=['421004'])

cols = [i for i in DataFrame.columns if i not in ['Date']]
for col in cols:
    DataFrame[col] = pd.to_numeric(DataFrame[col])

DataFrame.head()


,Date,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,425010,409025,423202CO,423203A,410005,water year
14906,1994-07-01,0.0,41.903,101.976,0.0,0.0,14.83,2605.98,124.930,306.87,313.926,361.634,13488.707,12101.159,0.0,0.0,3555.327,1994
14907,1994-07-02,0.0,41.953,99.745,0.0,0.0,21.05,2460.09,129.196,368.63,313.844,209.870,13350.562,12206.539,0.0,0.0,2956.510,1994
14908,1994-07-03,0.0,42.084,101.291,0.0,0.0,26.62,2043.70,136.214,385.78,314.913,105.297,13023.193,12062.432,0.0,0.0,5218.797,1994
14909,1994-07-04,0.0,41.479,100.029,0.0,0.0,25.43,1590.57,137.673,407.55,312.013,123.557,12510.113,10969.362,0.0,0.0,4276.486,1994
14910,1994-07-05,0.0,46.937,93.188,0.0,0.0,22.10,1243.94,198.576,428.06,311.891,327.890,12017.372,9927.890,0.0,0.0,3787.678,1994


In [0]:
#############
## Define Lyne and Hollick filter functions (see command 3 for further information on the functions)
#############

def LH(flow, alpha = 0.98, passes=3 , Nreflect=30):
  '''Lyne and Hollick filter takes a real flow timeseries and returns 
  the baseflow component of the flow as a pandas dataframe'''
  
  # make sure flow is numeric
  flow = pd.to_numeric(flow)
  
  def FirstPass(flow,alpha ):
    '''gets a series of actual flow, and alpha
    returns quickflow and baseflow'''
    
    Qf1 = pd.Series(np.zeros(len(flow)))
    Qf1[0] = flow[0]
    for i in range(1,len(flow)):
      Qf1[i] = alpha * Qf1[i-1] + 0.5 * (1+alpha)*(flow[i] - flow[i-1])
    
    Qb1 = np.where(Qf1 > 0, flow - Qf1, flow)

    return Qf1, Qb1
  
  def BackwardsPass(flowDF, alpha):
    '''Takes a dataframe of Quick and Base flow
    and returns Quick and Base flow'''
    Qq = flowDF['Quick']
    Qb = flowDF['Base']
    Qf2 = pd.Series(np.zeros(len(flowDF)))
    Qf2.iloc[-1] = Qb.iloc[-1]
    
    for i in range(len(flowDF)-2, 0, -1):
       Qf2[i] = alpha * Qf2[i+1] + 0.5 * (1 + alpha) * (Qb[i] - Qb[i+1])
        
    Qb2 = np.where(Qf2 > 0, Qb - Qf2, Qb)
      
    return  Qf2, Qb2
  
  def ForwardPass(flowDF, alpha):
    '''Takes in a streamflow dataframe,
    seperates into a streamflow and a baseflow element,
    returns these as dataframes'''
    Qq = flowDF['Quick']
    Qb = flowDF['Base']
    Qf2 = pd.Series(np.zeros(len(flowDF)))
    Qf2[0] = Qb[0]
    
    for i in range(1,len(flowDF)):
      Qf2[i] = alpha* Qf2[i-1] + 0.5 * (1 + alpha) * (Qb[i]-Qb[i-1])
    
    Qb2 = np.where(Qf2 > 0, Qb - Qf2, Qb)
    
    return Qf2, Qb2
  # reflection
 
  def BFIcalculate(flow, alpha, passes, Nreflect):
    Qrefelect = pd.Series(np.zeros(len(flow) + 2 * Nreflect)) 
    Qrefelect[:Nreflect] = flow.iloc[Nreflect :0:-1]
    Qrefelect[Nreflect :Nreflect + len(flow)] = flow
    Qrefelect[Nreflect + len(flow) :] = flow.iloc[len(flow)-2:len(flow)-32:-1]
    
    # first pass through the filter
    qf, qb = FirstPass(Qrefelect, alpha)
    
    
    npass = round((passes-1)/2)
    
    def MakeDF(a,b):
      d={"Quick": a, "Base": b}
      Dataframe = pd.DataFrame(d)
      return Dataframe
    
    firstpassDF = MakeDF(qf, qb )
    
    q,b =BackwardsPass(firstpassDF, alpha)
    DF = MakeDF(q,b)   
    
    for i in range(0,npass):
      
      q, b = ForwardPass(MakeDF(q,b),alpha)
      
      
      DF = MakeDF(q,b)
      
                
      q, b = BackwardsPass(DF,alpha)

      
    Base =  DF["Base"][Nreflect :Nreflect + len(flow)  ]
    Base[Base < 0] = 0
    
    
    BFI = Base.sum()/flow.sum()
    
    return BFI, Base
    
  
  BFI, Base= BFIcalculate(flow, alpha, passes , Nreflect) 
  
  return   BFI, Base.reset_index(drop= True)

In [0]:
# Takes streamflow dataframe and passes through baseflow filter (command 16) to return baseflow dataframe 

BaseflowDF = DataFrame[["Date", "water year"]].copy()

for gauge in DataFrame.columns.tolist()[1:-1]:
  flow = pd.to_numeric(DataFrame[gauge])
  
  BFI, DF =LH(flow, passes =3)
  
  LHdataframe = DF
  
  BaseflowDF[gauge] = LHdataframe.values
  
cols=[i for i in BaseflowDF.columns if i not in ["Date"]]
for col in cols:
  BaseflowDF[col]=pd.to_numeric(BaseflowDF[col])
  
BaseflowDF.head()

,Date,water year,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,425010,409025,423202CO,423203A,410005
14906,1994-07-01,1994,0.0,1.422007,2.088319,0.0,0.0,1.832482,92.445046,20.416994,40.453522,29.469241,23.469743,1340.493420,1149.644778,0.0,0.0,404.549400
14907,1994-07-02,1994,0.0,1.595435,2.408575,0.0,0.0,1.945475,99.912588,21.521594,42.790263,31.325206,24.979132,1417.371986,1214.984986,0.0,0.0,425.018833
14908,1994-07-03,1994,0.0,1.778146,2.755461,0.0,0.0,2.061571,108.001436,22.625798,45.160343,33.220797,26.507425,1495.960514,1282.109394,0.0,0.0,445.976384
14909,1994-07-04,1994,0.0,1.969671,3.114504,0.0,0.0,2.182216,116.540572,23.731155,47.549270,35.153602,28.018996,1576.029259,1350.792909,0.0,0.0,467.692684
14910,1994-07-05,1994,0.0,2.170021,3.468724,0.0,0.0,2.307169,125.341820,24.844260,49.912049,37.120891,29.527532,1657.302949,1420.609144,0.0,0.0,490.117885


In [0]:
# Getting the annual sum of the baseflow component for each catchment
# Getting the baseflow index (baseflow/total stream flow). A value closer to 1
# represents drier conditions (baseflow making up a greater portion of 
# total flow).

BFIDF = pd.DataFrame([])
for gauge in DataFrame.columns.tolist()[1:-1]:
    GaugeDF = pd.DataFrame(data=[], columns=['Water Year', gauge])
    for waterYear in BaseflowDF['water year'].unique():
        Base = BaseflowDF[BaseflowDF['water year'] == waterYear][gauge]
        flow = DataFrame[DataFrame['water year']
                            == waterYear][gauge]
        BFI = Base.sum() / flow.sum()

        StepDF = pd.DataFrame({'Water Year': [waterYear], gauge: [BFI]})
        GaugeDF = GaugeDF.append(StepDF)
    if ~BFIDF.empty:
        BFIDF = pd.concat([BFIDF, GaugeDF], axis=1)
    else:
        BFIDF = GaugeDF

BFIDF = BFIDF.loc[:, ~BFIDF.columns.duplicated()]


In [0]:
BFIDF = BFIDF.reset_index(drop=True)
BFIDF.head()


,Water Year,422015,418004,419021,425003,416001,406202,405232,421005,407202,425012,421090,425010,409025,423202CO,423203A,410005
0,1994,0.044135,0.112023,0.052602,0.062002,0.079225,0.153404,0.330848,0.605055,0.419051,0.481378,0.302928,0.606216,0.663296,0.071288,0.062216,0.564122
1,1995,0.072601,0.127540,0.079136,0.143128,0.093736,0.250412,0.345924,0.659565,0.550475,0.324767,0.533766,0.514282,0.603712,0.036901,0.038629,0.556648
2,1996,0.064209,0.203803,0.111372,0.276513,0.245509,0.152506,0.266125,0.665770,0.457598,0.277736,0.405335,0.463403,0.415346,0.044592,0.036500,0.543804
3,1997,0.229692,0.376918,0.226272,0.653080,0.511125,0.431045,0.693865,0.694668,0.520046,0.405041,0.464220,0.738099,0.699351,0.131563,0.093375,0.580755
4,1998,0.093821,0.178058,0.173360,0.173342,0.143751,0.400833,0.511580,0.350588,0.599052,0.255190,0.410852,0.589858,0.636080,0.160356,0.168707,0.587708


In [0]:
BFIDFresults = pd.melt(BFIDF, id_vars=['Water Year'],
                       value_vars=DataFrame.columns.tolist()[1:-1],
                       var_name='ID')
BFIDFmerged = BFIDFresults

BFIDFmerged.head()

,Water Year,ID,value
0,1994,422015,0.044135
1,1995,422015,0.072601
2,1996,422015,0.064209
3,1997,422015,0.229692
4,1998,422015,0.093821


## Compare pre and post Basin Plan
Compare the pre and post Basin Plan baseflow index using:
- Welsh's T-test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
- the KS two sample test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html)

In [0]:
def siteloop(ResultsDataFrame, ID, quiet=True):
  '''Takes in dataframe with baseflow index and date,
  filters the dataframe to pre and post basin plan periods,
  runs  Welsh's t test and ks two sample test on both periods,
  returns the results dataframe'''
  pre = np.array(ResultsDataFrame[(ResultsDataFrame['Water Year']
                 < 2012) & (ResultsDataFrame['ID'] == ID)]['value'])
  post = np.array(ResultsDataFrame[(ResultsDataFrame['Water Year']
                  >= 2012) & (ResultsDataFrame['ID'] == ID)]['value'])

  (ksStat, KsP) = scipy.stats.ks_2samp(pre, post)
  (tStat, tP) = scipy.stats.ttest_ind(pre, post, equal_var=False)

  Outcome = Significant(KsP, tStat, tP, alpha)

  if not quiet:
      print (ID, scipy.stats.ks_2samp(pre, post))
      print (ID, scipy.stats.ttest_ind(pre, post, equal_var=False))
    
  StepDataFrame = pd.DataFrame({
    "ID":[ID], 
    "Metric":["Baseflows"], 
    "Source":["Observed"],
    "Ks_2sampResult statistic":[ksStat], 
    "Ks_2sampResult pvalue":[KsP], 
    "Welch’s t-test statistic":[tStat], 
    "Welch’s t-test pvalue":[tP], 
    "Outcome":[Outcome]
     }) 
  
  return StepDataFrame


def Significant (Ksp, tStat, tP, alpha):
  '''Takes in results of statistical tests,
  compares the results of the two tests to an alpha value defined by the operator,
  returns the significance'''
  if ((Ksp < alpha) and (tStat <0) and (tP < alpha)):
    
    outcome = "Increased" 
  elif (tStat >0 and Ksp <alpha and tP < alpha):
    outcome = "Decreased" 
  elif (Ksp >alpha and tP > alpha):
    outcome = "Maintained" 
  elif (Ksp <alpha and tP > alpha):
    outcome = "Unsure - t-test failed" 
  else:
    outcome = "Unsure - ks-test failed"
  return outcome
  

##Selecting an Alpha 
With two tests with alphas set at 0.1, the probability of observing a false statistically significant results in both tests is 1%  

Typically, methods for dealing with multiple tests call for adjusting alpha in some way, however, these methods are designed for statistical investigations looking for a single significant result, ‘a discovery’. This is not the case in the application of two statistical tests looking for concurrent significant results.  

Setting alpha to 0.1 in both tests so that the chance of a false positive ‘increased’ or ‘decreased’ result is 1% is suitably rigorous and decidedly reasonable for the task at hand.

In [0]:
alpha = 0.1

StatsResults = pd.DataFrame(data=[],columns = [
  "ID", 
  "Metric", 
  "Source", 
  "Ks_2sampResult statistic", 
  "Ks_2sampResult pvalue", 
  "Welch’s t-test statistic", 
  "Welch’s t-test pvalue", 
  "Outcome"
  ])

for ID in BFIDFmerged["ID"].unique():

  StepDataFrame = siteloop(BFIDFmerged, ID) 
  StatsResults = StatsResults.append(StepDataFrame)

StatsResults

,ID,Metric,Source,Ks_2sampResult statistic,Ks_2sampResult pvalue,Welch’s t-test statistic,Welch’s t-test pvalue,Outcome
0,422015,Baseflows,Observed,0.468254,0.155244,1.614068,0.123736,Maintained
0,418004,Baseflows,Observed,0.500000,0.108534,-2.823623,0.011153,Unsure - ks-test failed
0,419021,Baseflows,Observed,0.246032,0.872231,-0.708852,0.493442,Maintained
0,425003,Baseflows,Observed,0.436508,0.216793,-1.037583,0.315482,Maintained
0,416001,Baseflows,Observed,0.373016,0.392099,-1.217915,0.255169,Maintained
0,406202,Baseflows,Observed,0.277778,0.759528,-0.026184,0.979443,Maintained
0,405232,Baseflows,Observed,0.388889,0.341464,1.057724,0.301200,Maintained
0,421005,Baseflows,Observed,0.222222,0.935775,0.036460,0.971605,Maintained
0,407202,Baseflows,Observed,0.412698,0.274040,-1.305831,0.210375,Maintained
0,425012,Baseflows,Observed,0.611111,0.025748,-2.442072,0.033270,Increased
